# Оператор мобильной связи «Мегалайн»

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.
Построим модель с максимально большим значением accuracy.

## 1. Откроем и изучим данные

In [1]:
import pandas as pd
import os
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.shape

(3214, 5)

Вывод:
- Данные состоят из 3214 строк и 5 столбцов
- Целевой признак - столбец is_ultra(качественный тип данных). Мы построим модель, которая будет присваивать значение 1 пользователям, кому стоит предложить сменить тариф на Ультра
- Перед нами стоит задача классификации

## 2. Поделим данные на части

In [5]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=1)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=1)

Все данные поделили на 60 и 40. Оставшиеся 40 поделили на две равные части. Таким образом получаем:
- Обучающеая выборка 60%
- Валидационная выборка 20%
- Тестовая выборка 20%

## 3. Исследуем модели

Возьмем 3 модели:
- Logistic Regression (Логическая регрессия)
- Decision Tree (Дерево решений)
- Random Forest (Случайный лес)

Обозначим целевые признаки 

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Функция для подсчета точности модели на валидационной выборке

In [7]:
def show_accuracy_score(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

### Logistic Regression (Логическая регрессия)

Так как для данной модели есть несколько разных параметров, создадим функцию и цикл для испытания всех параметров.

In [8]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

In [9]:
def logistic_regression(solver=solvers):
    model = LogisticRegression(solver=solver, random_state=1)
    model.fit(features_train, target_train)
    return show_accuracy_score(model)

In [10]:
for solver in solvers:
    print(solver, ': ', logistic_regression(solver=solver))

newton-cg :  0.7200622083981337
lbfgs :  0.7200622083981337
liblinear :  0.71850699844479
sag :  0.6640746500777605
saga :  0.6640746500777605


Лучший результат дают параметры newton-cg и lbfgs: 0.7200622083981337

### Decision Tree (Дерево решений)

In [11]:
for depth in range(1, 20, 2):
    model_tree = DecisionTreeClassifier(random_state=1, max_depth=depth)
    model_tree.fit(features_train, target_train)
    print('depth =', depth, ' \ accuracy =', show_accuracy_score(model_tree))

depth = 1  \ accuracy = 0.71850699844479
depth = 3  \ accuracy = 0.7713841368584758
depth = 5  \ accuracy = 0.7698289269051322
depth = 7  \ accuracy = 0.7776049766718507
depth = 9  \ accuracy = 0.7962674961119751
depth = 11  \ accuracy = 0.7869362363919129
depth = 13  \ accuracy = 0.7900466562986003
depth = 15  \ accuracy = 0.7667185069984448
depth = 17  \ accuracy = 0.7636080870917574
depth = 19  \ accuracy = 0.7325038880248833


Лучший результат при параметре глубины: depth = 9  \ accuracy = 0.7962674961119751. Чуть лучше предыдущего результата

### Random Forest (Случайный лес)

In [12]:
for estim in range(10, 101, 10):
    model_forest = RandomForestClassifier(n_estimators=estim, max_depth=9, random_state=1)
    model_forest.fit(features_train, target_train)
    print('n_estimators =', estim, ' / accuracy_score =', show_accuracy_score(model_forest))

n_estimators = 10  / accuracy_score = 0.7931570762052877
n_estimators = 20  / accuracy_score = 0.7916018662519441
n_estimators = 30  / accuracy_score = 0.7884914463452566
n_estimators = 40  / accuracy_score = 0.7884914463452566
n_estimators = 50  / accuracy_score = 0.7931570762052877
n_estimators = 60  / accuracy_score = 0.7916018662519441
n_estimators = 70  / accuracy_score = 0.7884914463452566
n_estimators = 80  / accuracy_score = 0.7900466562986003
n_estimators = 90  / accuracy_score = 0.7916018662519441
n_estimators = 100  / accuracy_score = 0.7916018662519441


Лучший результат при параметре количества деревьев и глубины 9(взяли с прошлого удачного показателя): depth = 9 / n_estimators = 10  / accuracy_score = 0.7931570762052877. Чуть хуже предыдущего результата

### Вывод:
Модели со следущими параметрами дали лучший результат среди других параметров:
- Logistic Regression(solver='newton-cg'): accuracy = 0.7200622083981337
- Decision Tree(depth = 9): accuracy = 0.7962674961119751
- Random Forest(depth = 9 / n_estimators = 10): accuracy = 0.7931570762052877

В итоге мы выбираем Дерево решений с параметром глубины 9.

## 4. Проверим модель на тестовой выборке

In [13]:
model = DecisionTreeClassifier(max_depth=9, random_state=1)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.7869362363919129

На тестовой выборке выбранная модель показывает точность близкую к валидационной, но не больше. Это означает отсутствие проблем с переобучение модели.

## 5. (бонус) Проверьте модели на адекватность

In [14]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [15]:
df_test['is_ultra'].value_counts()

0    446
1    197
Name: is_ultra, dtype: int64

In [16]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Доля большего класса тестовой выборки равна ~0.69 (Смарт); полученная нами модель имеет accuracy ~0.79. Таким образом, мы можем считать модель адекватной для использования.

